In [2]:
%pip install pulp

  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
Using cached PuLP-2.9.0-py3-none-any.whl (17.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pulp

In [9]:
# dict for the parameters [height, width] for each component
param_dict = {'chipset':[5,5], 'cpu':[3,3], 'coolfan':5, 'cmos':3, 'bios':[3,3], 'powercon':[2,1], 'usb':[2,1], 'ram':[2,10]}

# define the LP
problem = pulp.LpProblem('motherboard', pulp.LpMinimize)

# define variables
p = pulp.LpVariable.dicts('p',[1,2], cat='Continuous')
x = pulp.LpVariable.dicts('x',range(1, 12), lowBound=0, cat='Continuous')
print(x)
y = pulp.LpVariable.dicts('y',range(1,12), lowBound=0, cat='Continuous')
W = pulp.LpVariable('W', lowBound=0, cat='Continuous')
H = pulp.LpVariable('H', lowBound=0, cat='Continuous')

# objective function
problem += 2*W + 2*H

# hori constraints
problem += x[1] - p[1] >= 0
problem += x[1] + p[1] + param_dict['chipset'][1] + param_dict['coolfan'] <= x[3]
problem += x[4] + p[1] >= 0
problem += x[4] + p[1] + 2*param_dict['cmos'] <= x[5]
problem += x[5] + p[1] + param_dict['bios'][1] <= x[6]
problem += x[6] + p[1] + param_dict['powercon'][1] <= x[7]
problem += x[7] + p[1] + param_dict['usb'][1] <= x[8]
problem += x[3] + p[1] + param_dict['coolfan'] <= x[8]
problem += x[8] + p[2] + param_dict['ram'][1] <= x[9]
problem += x[9] + p[2] + param_dict['ram'][1] <= x[10]
problem += x[10] + p[2] + param_dict['ram'][1] <= x[11]
problem += x[11] + p[2] + param_dict['ram'][1] <= W

#vert constraints
problem += y[4] + p[1] + param_dict['cmos'] >= 0
problem += y[4] + p[1] + param_dict['cmos'] <= y[1]
problem += y[1] + p[1] + param_dict['chipset'][0] <= H
problem += y[5] + p[1] >= 0
problem += y[5] + p[1] + param_dict['bios'][0] <= y[1]
problem += y[6] + p[1] >= 0
problem += y[6] + p[1] + param_dict['powercon'][0] + param_dict['coolfan'] <= y[3]
problem += y[3] + p[1] + param_dict['coolfan'] <= H
problem += y[7] + p[1] >= 0
problem += y[7] + p[1] + param_dict['usb'][0] + param_dict['coolfan'] <= y[3]
problem += y[8] + p[1] >= 0
problem += y[8] + param_dict['ram'][0] <= H
problem += y[9] + p[1] >= 0
problem += y[9] + param_dict['ram'][0] <= H
problem += y[10] + p[1] >= 0
problem += y[10] + param_dict['ram'][0] <= H
problem += y[11] + p[1] >= 0
problem += y[11] + param_dict['ram'][0] <= H

#alignment constraints
x[2] + 0.5*param_dict['cpu'][1] == x[3]
x[3] + param_dict['coolfan'] == x[6]
y[2] + 0.5*param_dict['cpu'][0] == y[3]

# solve
problem.solve()

# create dictionary with the solutions
solution = {}
for var in problem.variables():
    solution[var.name] = var.varValue

print(W, H)

{1: x_1, 2: x_2, 3: x_3, 4: x_4, 5: x_5, 6: x_6, 7: x_7, 8: x_8, 9: x_9, 10: x_10, 11: x_11}
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jin/Documents/GitHub/adv-opti/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/gt/q99q3ns57l75nmgnyb_khwnm0000gn/T/cecd2affa9744fe99e3f6548f76b81e8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/gt/q99q3ns57l75nmgnyb_khwnm0000gn/T/cecd2affa9744fe99e3f6548f76b81e8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 35 COLUMNS
At line 114 RHS
At line 145 BOUNDS
At line 148 ENDATA
Problem MODEL has 30 rows, 24 columns and 76 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 11 (-19) rows, 8 (-16) columns and 28 (-48) elements
Perturbing problem by 0.001% of 2 - largest nonzero change 0.00010342582 ( 0.0051712911%) - largest zero change 8.3259078e-05
0  Obj 4.0002